<a href="https://colab.research.google.com/github/Hvaa1/Speech_to_text_and_Emotion_recognition/blob/main/PhoBertFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd "/content/drive/MyDrive/ColabNotebooks/EmotionRecognition"

/content/drive/MyDrive/ColabNotebooks/EmotionRecognition


In [ ]:
!pip install --upgrade transformers

In [ ]:
import transformers
print(transformers.__version__)
from transformers import TrainingArguments
print(TrainingArguments)


4.57.0
<class 'transformers.training_args.TrainingArguments'>


In [ ]:
from datasets import load_dataset
ds = load_dataset("tridm/UIT-VSMEC")
print(ds['train'].column_names)
print(ds["train"][1])

['Sentence', 'Emotion']
{'Sentence': 'cho đáng đời con quỷ . về nhà lôi con nhà mày ra mà đánh 😡', 'Emotion': 'Disgust'}


In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

def tokenize_batch(batch):
    return tokenizer(batch["Sentence"], padding="max_length", truncation=True, max_length=225)

ds = ds.map(tokenize_batch, batched=True)
ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/5548 [00:00<?, ? examples/s]

Map:   0%|          | 0/686 [00:00<?, ? examples/s]

Map:   0%|          | 0/693 [00:00<?, ? examples/s]

In [ ]:
print(ds['train'].column_names)

['Sentence', 'labels', 'input_ids', 'token_type_ids', 'attention_mask']


In [ ]:
label2id = {"Anger":0, "Disgust":1, "Fear":2, "Enjoyment":3, "Sadness":4, "Surprise":5, "Other":6}
id2label = {v:k for k,v in label2id.items()}

def encode_labels(example):
    example["labels"] = label2id[example["labels"]]
    return example

ds = ds.map(encode_labels)

Map:   0%|          | 0/5548 [00:00<?, ? examples/s]

Map:   0%|          | 0/686 [00:00<?, ? examples/s]

Map:   0%|          | 0/693 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

num_labels = 7
model = AutoModelForSequenceClassification.from_pretrained('vinai/phobert-base-v2', num_labels=num_labels)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    push_to_hub=False ,
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
)

trainer.train()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss
500,1.381800,1.182748
1000,0.933400,1.093946


TrainOutput(global_step=1041, training_loss=1.1417946646009696, metrics={'train_runtime': 455.4216, 'train_samples_per_second': 36.546, 'train_steps_per_second': 2.286, 'total_flos': 1094854250419200.0, 'train_loss': 1.1417946646009696, 'epoch': 3.0})

In [ ]:
import torch
import torch.nn.functional as F

labels_map = {
    0: "Anger", 1: "Disgust", 2: "Fear",
    3: "Enjoyment", 4: "Sadness", 5: "Surprise", 6: "Other"
}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def predict(text):
    encoding = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    encoding = {k: v.to(device) for k, v in encoding.items()}

    outputs = model(**encoding)
    probs = F.softmax(outputs.logits, dim=-1)
    pred = torch.argmax(probs, dim=-1).item()
    return labels_map[pred], probs.detach().cpu().numpy()
# Ví dụ
sentence = "Trời mưa làm tôi buồn quá"
emotion, probs = predict(sentence)
print("Text:", sentence)
print("Cảm xúc:", emotion)
print("Xác suất:", probs)


Text: Trời mưa làm tôi buồn quá
Cảm xúc: Sadness
Xác suất: [[0.02512745 0.03769794 0.04507938 0.03661628 0.7694341  0.02533494
  0.06070995]]


In [ ]:

trainer.save_model("./Phobert_ft1")
tokenizer.save_pretrained("./Phobert_ft1")


('./Phobert_ft1/tokenizer_config.json',
 './Phobert_ft1/special_tokens_map.json',
 './Phobert_ft1/vocab.txt',
 './Phobert_ft1/bpe.codes',
 './Phobert_ft1/added_tokens.json')